In [ ]:
pip install faiss-cpu scikit-learn

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/dinesh/Documents/vs code/LLM_Fine-tuning/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up directories and file paths
preprocessed_data_dir = "/home/dinesh/Downloads/dataset/processed-IN-Ext/"
train_file_A1 = os.path.join(preprocessed_data_dir, "full_summaries_A1.jsonl")

In [3]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load variables from .env into the notebook kernel
load_dotenv()

login(token=os.getenv("HF_TOKEN"))


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
lora_path = "/home/dinesh/Downloads/dataset/fine_tuned_lora_model"
base_model = "meta-llama/Llama-2-7b-hf"   # <-- must be the exact base you trained on

tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
# If your tokenizer has no pad token (common for Llama)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
)

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.92it/s]


In [5]:
# Load and preprocess the dataset
def load_dataset_for_retrieval(jsonl_file):
    """
    Load preprocessed data and extract judgments for retrieval.
    """
    with open(jsonl_file, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]

    # Extract judgments and deduplicate
    judgments = list(set(item["judgement"].strip() for item in data))
    return judgments

# Load judgments from the file and deduplicate
all_judgments = load_dataset_for_retrieval(train_file_A1)

In [6]:
# Convert judgments to TF-IDF vectors for retrieval
vectorizer = TfidfVectorizer()
judgment_vectors = vectorizer.fit_transform(all_judgments).toarray().astype(np.float32)

# Build FAISS index for efficient similarity search
index = faiss.IndexFlatL2(judgment_vectors.shape[1])  # L2 distance
index.add(judgment_vectors)

In [7]:
def retrieve_judgments(query, top_k=3):
    """
    Retrieve top-k judgments relevant to the query using FAISS.
    """
    query_vector = vectorizer.transform([query]).toarray().astype(np.float32)
    distances, indices = index.search(query_vector, top_k)
    return [all_judgments[i] for i in indices[0]]

### Implement the RAG pipeline

In [8]:
def pipeline(query, top_k=1, max_length=2048):
    retrieved_judgments = retrieve_judgments(query, top_k=top_k)
    summaries = []
    input_device = next(model.parameters()).device

    for judgment in retrieved_judgments:
        input_text = (
            "### Instruction: Summarize the following legal text.\n\n"
            f"### Input:\n{judgment.strip()}\n\n"
            "### Response:\n"
        )

        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            max_length=max_length,     # reduce from 4096
            truncation=True,
        )
        inputs = {k: v.to(input_device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.15,
                no_repeat_ngram_size=4,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                use_cache=True,
            )

        summary = tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True
        )
        summaries.append(summary)

    return summaries


In [9]:
if __name__ == "__main__":
    query = "lakshminarayana iyer"
    summaries = pipeline(query, top_k=1)

    print("Retrieved Summaries:")
    for i, summary in enumerate(summaries):
        print(f"Summary {i+1}:\n{summary}\n")

Retrieved Summaries:
Summary 1:
ating by way of gifts sales and exchanges a share of the properties left behind her by the husband.
she does not acquire such powers until she has exhausted her right to remain in occupation of the properties during her lifetime.

